In [31]:
import numpy as np
from tqdm import tqdm

import models
import preprocessing
import score_utils


In [3]:
scores = preprocessing.get_score_tensors()

Loading score tensors from disk


  0%|          | 0/5930 [00:00<?, ?it/s]

  1%|          | 48/5930 [00:00<00:12, 478.94it/s]

  2%|▏         | 109/5930 [00:00<00:10, 537.45it/s]

  3%|▎         | 155/5930 [00:00<00:11, 507.32it/s]

...

 96%|█████████▌| 5690/5930 [00:09<00:00, 591.71it/s]

 97%|█████████▋| 5752/5930 [00:09<00:00, 591.24it/s]

 98%|█████████▊| 5812/5930 [00:09<00:00, 590.26it/s]

 99%|█████████▉| 5879/5930 [00:09<00:00, 591.03it/s]

100%|██████████| 5930/5930 [00:10<00:00, 589.94it/s]

In [5]:
metas = preprocessing.get_metadata_tensors()

In [7]:
metas[0].shape

(5, 720, 18)

In [50]:
max_pitch = np.max([np.max(t) for t in scores])
min_pitch = np.min([np.min(t[t > 0]) for t in scores])

In [51]:
min_pitch

31.0

In [12]:
preprocessing.make_input_sequence(scores[0], metas[0], 0, max_pitch).shape

(720, 4, 32, 5, 3)

In [12]:
model = models.conv2dlstm_model()
model.load_weights('./weights/conv2dlstm.weights.best.hdf5')
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
conv_lst_m2d_1_input (InputLayer (None, None, 32, None 0                                            
____________________________________________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)      (None, None, 32, None 40448       conv_lst_m2d_1_input[0][0]       
____________________________________________________________________________________________________
max_pooling3d_1 (MaxPooling3D)   (None, None, 16, None 0           conv_lst_m2d_1[0][0]             
____________________________________________________________________________________________________
conv_lst_m2d_2 (ConvLSTM2D)      (None, 16, None, 64)  221440      max_pooling3d_1[0][0]            
___________________________________________________________________________________________

In [180]:
score_length = 4 * 16  # let's start with 4 bars
score_length

64

In [181]:
# The model is conditioned on a fixed beat pattern
beats = np.zeros((score_length, 16))
for i in range(score_length):
    beats[i, i % 16] = 1
beats

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [182]:
# Let's first try a score with 5 voices
n_voices = 5
sequence_steps = 4
window_size = 32
gibbs_steps = 100


In [386]:
def sample_softmax(preds, temperature=1.0):
    # taken from https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py
    # helper function to sample an index from a probability array
    preds = preds.astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [642]:
def sample_softmax_multi(preds, temperature=1.0):
    # taken from https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py
    # helper function to sample an index from a probability array
    preds = preds.astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds, axis=1).reshape((-1, 1))
    probas = np.apply_along_axis(lambda pred: np.random.multinomial(1, pred, 1), 1, preds)
    return probas.argmax(axis=1)

In [643]:
score = np.zeros((n_voices, score_length))
meta = np.zeros((n_voices, score_length, 18))
for i in range(n_voices):
    meta[i, :, 2:] = beats

In [644]:
def sample_1(n_voices, score_length, gibbs_steps, temperature):
    score = np.zeros((n_voices, score_length))
    meta = np.zeros((n_voices, score_length, 18))
    beats = np.zeros((score_length, 16))
    for i in range(score_length):
        beats[i, i % 16] = 1
    for i in range(n_voices):
        meta[i, :, 2:] = beats
    for step in tqdm(range(gibbs_steps)):
        for time_step in range(score_length):
            for voice in range(n_voices):
                input_ = preprocessing.make_input_sequence(score, meta, voice, max_pitch)
                note, slur = model.predict([
                    input_[time_step:time_step+1], beats[time_step:time_step+1]])
                sampled_note = sample_softmax(note[0], temperature)
                slur_proba = np.squeeze(np.array([1 - slur[0], slur[0]]))
                sampled_slur = sample_softmax(slur_proba, temperature)
                if sampled_note > 0:
                    score[voice, time_step] = sampled_note + min_pitch
                else:
                    meta[voice, time_step, score_utils.idx_rest] = 1
                meta[voice, time_step, score_utils.idx_slur] = sampled_slur
    return score, meta

In [645]:
def sample_2(n_voices, score_length, gibbs_steps, temperature):
    score = np.zeros((n_voices, score_length))
    meta = np.zeros((n_voices, score_length, 18))
    beats = np.zeros((score_length, 16))
    for i in range(score_length):
        beats[i, i % 16] = 1
    for i in range(n_voices):
        meta[i, :, 2:] = beats
    for step in tqdm(range(gibbs_steps)):
        for voice in range(n_voices):
            input_ = preprocessing.make_input_sequence(score, meta, voice, max_pitch)
            note, slur = model.predict([
                input_, beats])
            sampled_notes = sample_softmax_multi(note, temperature)
            slur_proba = np.hstack([1 - slur, slur])
            sampled_slurs = sample_softmax_multi(slur_proba, temperature)
            
            score[voice] = sampled_notes + ((sampled_notes > 1) * min_pitch)
            meta[voice, :, score_utils.idx_rest] = sampled_notes == 0
            meta[voice, :, score_utils.idx_slur] = sampled_slurs
    return score, meta

In [651]:
score, meta = sample_2(5, 32, 25, 1)

  0%|          | 0/25 [00:00<?, ?it/s]

  4%|▍         | 1/25 [00:01<00:42,  1.79s/it]

/Users/ben.cohen/miniconda3/envs/palestrinet/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


  8%|▊         | 2/25 [00:03<00:38,  1.67s/it]

 12%|█▏        | 3/25 [00:04<00:35,  1.61s/it]

...

 84%|████████▍ | 21/25 [00:26<00:04,  1.25s/it]

 88%|████████▊ | 22/25 [00:27<00:03,  1.24s/it]

 92%|█████████▏| 23/25 [00:28<00:02,  1.24s/it]

 96%|█████████▌| 24/25 [00:29<00:01,  1.24s/it]

100%|██████████| 25/25 [00:31<00:00,  1.24s/it]

array([[  0.,   0.,   0.,   0.,  49.,   0.,  49.,   0.,   0.,  49.,  56.,
         70.,   0.,   0.,   0.,   0.,   0.,  49.,  63.,  64.,  63.,  70.,
          0.,   0.,  51.,  50.,  49.,  51.,  47.,  49.,  52.,  51.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,  44.,  61.,   0.,   0.,
          0.,   0.,  47.,  58.,  69.,  68.,  71.,  68.,  58.,  68.,  63.,
          0.,   0.,  45.,  46.,   0.,  44.,  56.,  47.,  57.,  51.],
       [  0.,   0.,  54.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  44.,
          0.,   0.,   0.,  42.,  52.,  63.,  59.,   0.,   0.,   0.,   0.,
          0.,  44.,  45.,  44.,  44.,  47.,  44.,  47.,  44.,  54.],
       [  0.,   0.,   0.,   0.,   0.,  49.,  63.,   0.,  51.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,  54.,   0.,   0.,   0.,
          0.,  44.,  47.,  47.,  49.,  37.,   0.,  42.,  44.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,  51.,  54.,  46.,  45.,
         45.,   0.,   0.,  45.,   0.,   0.,   0.,   0.,   

In [654]:
music = score_utils.tensor_to_score(score, meta)

In [655]:
music.write('midi')

'/var/folders/tz/1572h57x6blgvbx33p6nyg3xbw4mw_/T/music21/tmp1r8nx5y_.mid'

In [119]:
score.shape

(5, 256)

In [122]:
meta[2, :, 0]

array([ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,
        1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,
        1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,
        1.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0